## Mask Detection in Videos 

In [3]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

## Set the locations for the face detection dependencies and mask detection model

In [4]:
args = {}
args["face"] = "face_detector"
args["model"] = "mask_detector.model"
args["confidence"] = 0.4

## Load the prototxt file and weights required by the face detector caffemodel

In [5]:
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([args["face"], "deploy.prototxt"])
weightsPath = os.path.sep.join([args["face"],
	"res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

[INFO] loading face detector model...


## Load the face mask detection model

In [6]:
print("[INFO] loading face mask detector model...")
maskNet = load_model(args["model"])

[INFO] loading face mask detector model...


## Define a function to detect faces and predict the presence masks on them

In [7]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	faceNet.setInput(blob)
	detections = faceNet.forward()

	faces = []
	locs = []
	preds = []

	for i in range(0, detections.shape[2]):
		confidence = detections[0, 0, i, 2]

		if confidence > args["confidence"]:
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			faces.append(face)
			locs.append((startX, startY, endX, endY))

	if len(faces) > 0:

		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	return (locs, preds)

## Detect masks in the frames extracted from the input video

In [13]:
print("[INFO] loading input video...")
input_name = "input1"
input_file =  "dataset/videos/" + input_name + ".mov"
cam = cv2.VideoCapture(input_file)
fps = cam.get(cv2.CAP_PROP_FPS)
count = 0
if not os.path.isdir('Frames/' + input_name):
	os.mkdir('Frames/' + input_name)

while True:
	sucess, frame = cam.read()
	if frame is None:
		break
	frame = imutils.resize(frame, width=400)
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	for (box, pred) in zip(locs, preds):

		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	cv2.imwrite("Frames/{}/{}_frame.jpg".format(input_name, count), frame)
	count += 1

[INFO] loading input video...


## Generate the output video from the detection frames

In [11]:
def generate_video(): 
    image_folder = os.path.join('Frames', input_name) 
    video_file = 'outputs/videos/output.mp4'
    print(image_folder)

    if os.path.exists(video_file):
        os.remove(video_file)

    images = [img for img in os.listdir(image_folder)
              if img.endswith(".jpg") or
                 img.endswith(".jpeg") or
                 img.endswith(".png")] 

    images.sort(key = lambda f: int(f.replace('_frame.jpg','')))
    images.sort(key = lambda f: int(f.replace('_frame.jpeg','')))
    images.sort(key = lambda f: int(f.replace('_frame.png','')))

    print(len(images))

    frame = cv2.imread(os.path.join(image_folder, images[0])) 
    height, width, layers = frame.shape  

    img_array = []
    out = cv2.VideoWriter(video_file,cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

    for filename in images:
        img_array = cv2.imread(os.path.join(image_folder, filename))
        out.write(img_array)

    out.release()
        

In [12]:
generate_video()

Frames/input1
0


IndexError: list index out of range